<a href="https://colab.research.google.com/github/bhavesh-choudhary1/machine_learning_concept/blob/main/Combine_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.drop(columns=['Name','Ticket','Cabin'],inplace=True)

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(2)
memory usage: 62.8+ KB


In [ ]:
df.isnull().mean()*100

,0
PassengerId,0.000000
Survived,0.000000
Pclass,0.000000
Sex,0.000000
Age,19.865320
SibSp,0.000000
Parch,0.000000
Fare,0.000000
Embarked,0.224467


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [ ]:
x_train.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,332,1,male,45.5,0,0,28.5000,S
733,734,2,male,23.0,0,0,13.0000,S
382,383,3,male,32.0,0,0,7.9250,S
704,705,3,male,26.0,1,0,7.8542,S
813,814,3,female,6.0,4,2,31.2750,S


In [ ]:
x_train.shape,y_train.shape

((712, 8), (712,))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer

In [ ]:
x_train.head(2)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,332,1,male,45.5,0,0,28.5,S
733,734,2,male,23.0,0,0,13.0,S


In [ ]:
num_pipe = Pipeline([
    ('impute_age', SimpleImputer()),
    ('ss', StandardScaler())
])


In [ ]:
cat_pipe = Pipeline([
    ('impute_embarked', SimpleImputer(strategy='most_frequent')),
    ('encode_Sex_Embarked', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

In [ ]:
apply_col = ColumnTransformer(transformers=[
    ('num_pipe',num_pipe,[3]),
    ('cat_pipe',cat_pipe,[2,7])
],remainder='passthrough')

In [ ]:
combine_pipeline = Pipeline([
    ('apply_col',apply_col),
    ('model',LogisticRegression())
])

In [ ]:
apply_col

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_pipe',
                                 Pipeline(steps=[('impute_age',
                                                  SimpleImputer()),
                                                 ('ss', StandardScaler())]),
                                 [3]),
                                ('cat_pipe',
                                 Pipeline(steps=[('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encode_Sex_Embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 [2, 7])])

In [ ]:
combine_pipeline.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('apply_col',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num_pipe',
                                                  Pipeline(steps=[('impute_age',
                                                                   SimpleImputer()),
                                                                  ('ss',
                                                                   StandardScaler())]),
                                                  [3]),
                                                 ('cat_pipe',
                                                  Pipeline(steps=[('impute_embarked',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encode_Sex_Embarked',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  [2, 7])])),
                ('model', LogisticRegression())])

In [ ]:
y_pred = combine_pipeline.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_pred)

0.7988826815642458